In [42]:
input_sentence = "two large pizzas with ham and one bottle of diet coke and one small pizza with mushrooms"
model_1_prediction = ['B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'B-DRINKORDER', 'I-DRINKORDER', 'I-DRINKORDER', 'I-DRINKORDER', 'I-DRINKORDER','O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER','I-PIZZAORDER']
model_2_prediction = ['B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING', 'O', 'B-NUMBER', 'B-CONTAINERTYPE', 'O', 'B-DRINKTYPE', 'I-DRINKTYPE','O', 'B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING']


input_sentence2="i want a large onion and pepperoni pizza with extra cheese"
model_1_prediction2 = ['O','O','B-NUMBER','B-SIZE','B-TOPPING','O','B-TOPPING','O','O','B-QUANTITY','B-TOPPING']
model_2_prediction2 = ['O','O','B-PIZZAORDER','I-PIZZAORDER','I-PIZZAORDER','I-PIZZAORDER','I-PIZZAORDER','I-PIZZAORDER','I-PIZZAORDER','I-PIZZAORDER','I-PIZZAORDER']

def map_to_json(input_sentence, model_1_prediction, model_2_prediction):
    words = input_sentence.split()
    order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    current_pizza = {}
    current_drink = {}
    
    for i, (word, tag1, tag2) in enumerate(zip(words, model_1_prediction, model_2_prediction)):
        if tag1 == 'B-PIZZAORDER':
            if current_pizza:
                order["ORDER"]["PIZZAORDER"].append(current_pizza)
            current_pizza = {"AllTopping": []}
        elif tag1 == 'B-DRINKORDER':
            if current_drink:
                order["ORDER"]["DRINKORDER"].append(current_drink)
            current_drink = {}
        
        if tag2 == 'B-NUMBER':
            if tag1 == 'B-PIZZAORDER' or tag1 == 'I-PIZZAORDER':
                current_pizza["NUMBER"] = word
            elif tag1 == 'B-DRINKORDER' or tag1 == 'I-DRINKORDER':
                current_drink["NUMBER"] = word
        elif tag2 == 'B-SIZE':
            if tag1 == 'B-PIZZAORDER'or tag1 == 'I-PIZZAORDER':
                current_pizza["SIZE"] = word
            elif tag1 == 'B-DRINKORDER' or tag1 == 'I-DRINKORDER':
                current_drink["SIZE"] = word
        elif tag2 == 'B-STYLE':
            if tag1 == 'B-PIZZAORDER'or tag1 == 'I-PIZZAORDER':
                current_pizza["STYLE"] = word
            elif tag1 == 'B-DRINKORDER' or tag1 == 'I-DRINKORDER':
                current_drink["STYLE"] = word
        elif tag2 == 'B-TOPPING':
            current_pizza["AllTopping"].append({"NOT": False, "Quantity": None, "Topping": word})
        elif tag2 == 'B-DRINKTYPE':
            current_drink["DRINKTYPE"] = word
        elif tag2 == 'I-DRINKTYPE':
            current_drink["DRINKTYPE"] += " " + word
        elif tag2 == 'B-CONTAINERTYPE':
            current_drink["CONTAINERTYPE"] = word
    
    if current_pizza:
        order["ORDER"]["PIZZAORDER"].append(current_pizza)
    if current_drink:
        order["ORDER"]["DRINKORDER"].append(current_drink)
    
    return order

# order_json = map_to_json(input_sentence, model_1_prediction, model_2_prediction)
# print(order_json)
order_json2 = map_to_json(input_sentence2, model_1_prediction2, model_2_prediction2)
print(order_json2)

{'ORDER': {'PIZZAORDER': [], 'DRINKORDER': []}}


In [43]:
# I would like to order two large chicago style pizzas with extra cheese, pepperoni,  no onions and bacon, 
# and three small thin crust pizzas with a lot of olives and ham. I also want five large bottles of coke and two medium cans of diet pepsi.

In [45]:
import json

formatted_order_json = json.dumps(order_json2, indent=4)
print(formatted_order_json)


{
    "ORDER": {
        "PIZZAORDER": [],
        "DRINKORDER": []
    }
}


In [ ]:
{
    "ORDER": {
      "PIZZAORDER": [
        {
          "NUMBER": "two",
          "SIZE": "large",
          "STYLE": "chicago style",
          "AllTopping": [
            {
              "NOT":False,
              "Quantity": "extra",
              "Topping": "cheese"
            },
            {
              "NOT": False,
              "Quantity": None,
              "Topping": "pepperoni"
            },
            {
              "NOT": True,
              "Quantity": None,
              "Topping": "onions"
            },
             {
              "NOT": True,
              "Quantity": None,
              "Topping": "bacon"
            }
          ]
        },
        {
          "NUMBER": "three",
          "SIZE": "small",
          "STYLE": "thin crust",
          "AllTopping": [
            {
              "NOT": False,
              "Quantity": "a lot",
              "Topping": "olives"
            },
            {
              "NOT": False,
              "Quantity": None,
              "Topping": "ham"
            }
          ]
        }
      ],
      "DRINKORDER": [
        {
          "NUMBER": "five",
          "SIZE": "large",
          "DRINKTYPE": "coke",
          "CONTAINERTYPE": "bottle"
        },
        {
          "NUMBER": "two",
          "SIZE": "medium",
          "DRINKTYPE": "diet pepsi",
          "CONTAINERTYPE": "can"
        }
      ]
    }
  }

In [39]:
#(ORDER (PIZZAORDER (NUMBER two) (SIZE large) pizzas with (TOPPING ham) ) and (DRINKORDER (NUMBER one) (DRINKTYPE diet coke)))
def parse_order(input_str):
    # Initialize the order structure
    order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    
    # Remove parentheses and split into components
    parts = input_str.replace('(', '').replace(')', '').split()
    
    current_item = {}
    current_type = None
    toppings = []
    complex_topping = None
    
    i = 0
    while i < len(parts):
        if parts[i] == 'ORDER':
            i += 1
            continue
        elif parts[i] == 'PIZZAORDER':
            if current_item and toppings:
                current_item["AllTopping"].extend(toppings)
                order["ORDER"][current_type].append(current_item)
            current_item = {"AllTopping": []}
            current_type = "PIZZAORDER"
            toppings = []
            i += 1
        elif parts[i] == 'DRINKORDER':
            if current_item and toppings:
                current_item["AllTopping"].extend(toppings)
                order["ORDER"][current_type].append(current_item)
            current_item = {}
            current_type = "DRINKORDER"
            toppings = []
            i += 1
        elif parts[i].startswith('NUMBER'):
            current_item["NUMBER"] = parts[i + 1]
            i += 2
        elif parts[i].startswith('SIZE'):
            current_item["SIZE"] = parts[i + 1]
            i += 2
        elif parts[i].startswith('TOPPING'):
            toppings.append({
                "NOT": False,
                "Quantity": None,
                "Topping": parts[i + 1]
            })
            i += 2
        elif parts[i].startswith('COMPLEX_TOPPING'):
            complex_topping = {}
            i += 1
        elif parts[i].startswith('QUANTITY') and complex_topping is not None:
            complex_topping["Quantity"] = parts[i + 1]
            i += 2
        elif parts[i].startswith('TOPPING') and complex_topping is not None:
            toppings.append({
                "NOT": False,
                "Quantity": complex_topping.get("Quantity"),
                "Topping": parts[i + 1]
            })
            complex_topping = None
            i += 2
        elif parts[i].startswith('DRINKTYPE'):
            current_item["DRINKTYPE"] = ' '.join(parts[i + 1:i + 3]) if parts[i + 2:i + 3] else parts[i + 1]
            i += 2
        else:
            i += 1
            
    if current_item:
        if toppings and current_type == "PIZZAORDER":
            current_item["AllTopping"].extend(toppings)
        order["ORDER"][current_type].append(current_item)
        
    return order

# Test the function
test_input = "(ORDER i want (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING onion ) and (TOPPING pepperoni ) pizza with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) )"
result = parse_order(test_input)
print(json.dumps(result, indent=2))

    


{
  "ORDER": {
    "PIZZAORDER": [
      {
        "AllTopping": [
          {
            "NOT": false,
            "Quantity": null,
            "Topping": "onion"
          },
          {
            "NOT": false,
            "Quantity": null,
            "Topping": "pepperoni"
          },
          {
            "NOT": false,
            "Quantity": null,
            "Topping": "cheese"
          }
        ],
        "NUMBER": "a",
        "SIZE": "large"
      }
    ],
    "DRINKORDER": []
  }
}
